# Author-Reviewer Agentic Demo

## Load the packages

In [ ]:
#r "nuget: dotenv.net"
#r "nuget: Microsoft.SemanticKernel, 1.55.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.54.0-alpha"
#r "nuget: AutoGen, 0.2.3"
#r "nuget: AutoGen.SourceGenerator, 0.2.3"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using Azure.AI.OpenAI;
using dotenv.net;

### Load the environment variables

In [ ]:
DotEnv.Load();

var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var deployment = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");

### Get an OpenAIClient

In [ ]:
AzureOpenAIClient GetAzureOpenAIClient()
{                
    if (string.IsNullOrEmpty(apiKey) || string.IsNullOrEmpty(endpoint))
    {
        throw new InvalidOperationException("Azure OpenAI configuration is not properly set.");
    }

    var client = new AzureOpenAIClient(
        new Uri(endpoint),
        new System.ClientModel.ApiKeyCredential(apiKey));

    return client;
}

### Get an OpenAI client

In [ ]:
var client = GetAzureOpenAIClient();

### Configure the author

In [ ]:
var author = new OpenAIChatAgent(
chatClient: client.GetChatClient(deploymentName: deployment),
name: "author",
temperature:1.0f,
systemMessage: "You are a technical document writer. Your task is to write a one page technical document the given a topic.")
    .RegisterMessageConnector()
    .RegisterMiddleware(
        async (messages, option, agent, _) =>
        {
            var reply = await agent.GenerateReplyAsync(messages, option);

            //if (reply.GetContent()?.ToLower().Contains("complete") is true)
            //{
            //    Console.WriteLine("Teacher has completed the conversation.");
            //    
            //    return new TextMessage(
            //        Role.Assistant,
            //        GroupChatExtension.TERMINATE,
            //        from: reply.From
            //    );
            //}

            return reply;
        })
    .RegisterPrintMessage();

### Configure the reviewer

In [ ]:
var reviewer = new OpenAIChatAgent(
chatClient: client.GetChatClient(deploymentName: deployment),
name: "reviewer",
temperature:0.1f,
systemMessage: "You are a technical document reviewer who provides feedback. Your task is to review the provided technical document and provide improvement suggestions including at least one edge case. When the document includes the feedback revisions, say 'TERMINATE' in your response")
    .RegisterMessageConnector()
    .RegisterMiddleware(
        async (messages, option, agent, _) =>
        {
            var reply = await agent.GenerateReplyAsync(messages, option);

            if (reply.GetContent()?.ToLower().Contains("TERMINATE") is true)
            {
                Console.WriteLine("Teacher has completed the conversation.");
                
                return new TextMessage(
                    Role.Assistant,
                    GroupChatExtension.TERMINATE,
                    from: reply.From
                );
            }

            return reply;
        })
    .RegisterPrintMessage();

### Start a conversation

In [ ]:
var conversation = await reviewer.InitiateChatAsync(
            receiver: author,
            message: @"prompt engineering.",
            maxRound: 6
        );

conversation